In [ ]:
# initial settings
settingsFile = 'settings.cfg'


recipeList = ['uves_cal_mbias','uves_cal_predict','uves_cal_mflat','uves_cal_orderpos','uves_cal_wavecal','uves_obs_scired', 'uves_obs_scired']
clpRecipes = {}
clpResults = {}

In [ ]:
# imports
import cpl,os, sys
from astropy.io import fits
import ConfigParser

In [ ]:
# Functions
def fnSetRecipe(recipe, customConfig = ''):
    '''
    Function that will initialise a cpl recipe instance. If a custom config file has been given, values in this config 
    file will be be set on the recipe
    '''
    cplRecipe = cpl.Recipe(recipe)
    
    # parameters
    for param in cplRecipe.param:
        try:
            param.value = customConfig.get(recipe,param.name)
        except:
            param.value = param.default
        param.debug = False

    return cplRecipe

#-----------------------------------------------------------------------------------------------------------------------
def fngetRecipeFileList(recipeName = '', dataList= '', keyword= ''):
    return [filename for filename in dataList \
                    if keyword.lower() in fits.getheader(filename)['HIERARCH ESO DPR TYPE'].lower()]


#-----------------------------------------------------------------------------------------------------------------------
def fnUpdateProductList(baseFolder, productList = {}):
    fileList =[[filenameFull, fits.getheader(filenameFull)['HIERARCH ESO PRO CATG'] ] \
                      for root, dirs, files in os.walk(baseFolder, topdown=False) \
                      for filenameFull in [os.path.join(root, filename) for filename in files] \
            if filename.endswith('.fits')]
    #, fits.getheader(filename)['HIERARCH ESO PRO CATG']
    categoryList = list(set([filename[1] for filename in fileList]))
    
    
    productList.update({category: [filename[0] for filename in fileList if category in filename[1] ] \
                   for category in categoryList})
    
    return productList


    
    

In [ ]:
# Parse Settings
cfgFileName = './{}'.format(settingsFile)
cfgFile = ConfigParser.RawConfigParser(allow_no_value=True)
cfgFile.read(cfgFileName)

In [ ]:
# initialise esorex
cpl.esorex.init()

In [ ]:
# define folders

# data
baseDataFolder_raw = os.path.join(cfgFile.get('global','workPath').strip("'"),'RawData',cfgFile.get('global','workDir').strip("'"))
baseDataFolder_reduced = os.path.join(cfgFile.get('global','workPath').strip("'"),'reduced',cfgFile.get('global','workDir').strip("'"))

# standart calibrations
baseCalibrations_raw = os.path.join(cfgFile.get('global','workPath'),'RawData/UVESCalibrationFiles/')

# recipe folders
outputFolders = {recipe:os.path.join(baseDataFolder_reduced,recipe) for recipe in recipeList}
tmpFolder = os.path.join(baseDataFolder_reduced,'tmp')


In [ ]:
# defining keywords per recipe
# http://www.eso.org/observing/dfo/quality/UVES/pipeline/recipe_calib.html

recipeKeywords = {}
recipeKeywords['uves_cal_mbias']    = {'DPR_CATG': 'CALIB', 'DPR_TYPE': 'BIAS', \
                                       'TAG': 'BIAS_{}'.format(cfgFile.get('global','arm')),  'PRO_CATG': ['MASTER_BIAS']}

recipeKeywords['uves_cal_predict']  = {'DPR_CATG': 'CALIB', 'DPR_TYPE': 'FMTCHK',   \
                                       'TAG': 'ARC_LAMP_FORM_{}'.format(cfgFile.get('global','arm')), 'PRO_CATG': ['LINE_TABLE', 'ORDER_TABLE', 'BACKGROUND_TABLE']}

recipeKeywords['uves_cal_orderpos'] = {'DPR_CATG': 'CALIB', 'DPR_TYPE': 'ORDERDEF', \
                                       'TAG': 'ORDER_FLAT_{}'.format(cfgFile.get('global','arm')),    'PRO_CATG': []}

recipeKeywords['uves_cal_mflat']    = {'DPR_CATG': 'CALIB', 'DPR_TYPE': 'FLAT',     \
                                       'TAG': 'FLAT_{}'.format(cfgFile.get('global','arm')), 'PRO_CATG': []}

recipeKeywords['uves_cal_wavecal']  = {'DPR_CATG': 'CALIB', 'DPR_TYPE': 'WAVE',     \
                                       'TAG': 'ARC_LAMP_{}'.format(cfgFile.get('global','arm')), 'PRO_CATG': []}

recipeKeywords['uves_obs_scired']   = {'DPR_CATG': 'SCIENCE', 'DPR_TYPE': 'OBJECT', \
                                       'TAG': 'SCIENCE_{}'.format(cfgFile.get('global','arm')),'PRO_CATG': []} 

# standard calibrations
calibrations_fileList = {}
calibrations_fileList['LINE_REFER_TABLE'] = [os.path.join(baseCalibrations_raw, 'thargood_3.fits')]
calibrations_fileList['LINE_INTMON_TABLE'] = [os.path.join(baseCalibrations_raw, 'thar_bright.fits')]


In [ ]:
# get filelists
# data
rawData_fileList = [os.path.join(baseDataFolder_raw, filename) for filename in os.listdir(baseDataFolder_raw) if filename.endswith('.fits')]

# Recipe file lists:
recipeFileLists = {}



In [ ]:
# 1) create master bias -     uves_cal_mbias 

recipe = 'uves_cal_mbias'

# get input file list
recipeFileLists[recipe] = fngetRecipeFileList(recipeName = recipe, \
                                              dataList = rawData_fileList, \
                                              keyword = recipeKeywords[recipe]['DPR_TYPE'])

# setup recipe "uves_cal_mbias"
clpRecipes[recipe] = fnSetRecipe(recipe,cfgFile)

# run recipe "uves_cal_predict"
clpResults[recipe] = clpRecipes[recipe](raw = {recipeKeywords[recipe]['TAG']:recipeFileLists[recipe]}, \
                                        output_dir = outputFolders[recipe], tmp_dir = tmpFolder)

clpResults[recipe] = None

print '{} complete.'.format(recipe)

In [ ]:
# 2) create guess order and line tables - uves_cal_predict
recipe = 'uves_cal_predict'

# get input file list
recipeFileLists[recipe] = fngetRecipeFileList(recipeName = recipe,\
                                              dataList = rawData_fileList,\
                                              keyword = recipeKeywords[recipe]['DPR_TYPE'])

#calibrations_fileList['MASTER_BIAS']                  = [os.path.join(outputFolders['uves_cal_mbias'], filename) \
#                                                   for filename in os.listdir(outputFolders['uves_cal_mbias'])\
#                                                  if 'MASTER_BIAS'.lower() in fits.getheaderos.path.join(outputFolders['uves_cal_mbias'], filename))['HIERARCH ESO PRO CATG'].lower()]
# update calibration lists
calibrations_fileList.update(fnUpdateProductList(baseDataFolder_reduced, productList = calibrations_fileList))

# setup recipe "uves_cal_predict"
clpRecipes[recipe] = fnSetRecipe(recipe,cfgFile)

# run recipe "uves_cal_predict"
#clpResults[recipe] = clpRecipes[recipe](calibrations_fileList['MASTER_BIAS'], calib = calibrations_fileList, logger=None)


clpResults[recipe] = clpRecipes[recipe](raw = {recipeKeywords[recipe]['TAG']:recipeFileLists[recipe]}, \
                                        calib = calibrations_fileList, output_dir = outputFolders[recipe], tmp_dir = tmpFolder)

#clpResults[recipe] = None

print '{} complete.'.format(recipe)

In [ ]:
# 3) create order table from order guess table      uves_cal_orderpos
recipe = 'uves_cal_orderpos'

# get input file list
recipeFileLists[recipe] = fngetRecipeFileList(recipeName = recipe,\
                                              dataList = rawData_fileList,\
                                              keyword = recipeKeywords[recipe]['DPR_TYPE'])


# update calibration lists
calibrations_fileList.update(fnUpdateProductList(baseDataFolder_reduced, productList = calibrations_fileList))

# setup recipe "uves_cal_orderpos"
clpRecipes[recipe] = fnSetRecipe(recipe,cfgFile)

# run recipe "uves_cal_predict"
clpResults[recipe] = clpRecipes[recipe](raw = {recipeKeywords[recipe]['TAG']:recipeFileLists[recipe]}, calib = calibrations_fileList, \
                                        output_dir = outputFolders[recipe], tmp_dir = tmpFolder)

clpResults[recipe] = None

print '{} complete.'.format(recipe)


In [ ]:
# 4) create master flat - uves_cal_mflat

# setup recipe "uves_cal_mflat"
recipe = 'uves_cal_mflat'

# get input file list
recipeFileLists[recipe] = fngetRecipeFileList(recipeName = recipe,\
                                              dataList = rawData_fileList,\
                                              keyword = recipeKeywords[recipe]['DPR_TYPE'])


# update calibration lists
calibrations_fileList.update(fnUpdateProductList(baseDataFolder_reduced, productList = calibrations_fileList))

# setup recipe 
clpRecipes[recipe] = fnSetRecipe(recipe,cfgFile)

# run recipe "uves_cal_predict"
clpResults[recipe] = clpRecipes[recipe](raw = {recipeKeywords[recipe]['TAG']:recipeFileLists[recipe] }, calib = calibrations_fileList, \
                                        output_dir = outputFolders[recipe], tmp_dir = tmpFolder)

clpResults[recipe] = None
print '{} complete.'.format(recipe)



In [ ]:
# 5) create line calibration table from guess table - uves_cal_wavecal

recipe = 'uves_cal_wavecal'

# get input file list
recipeFileLists[recipe] = fngetRecipeFileList(recipeName = recipe,\
                                              dataList = rawData_fileList,\
                                              keyword = recipeKeywords[recipe]['DPR_TYPE'])


# update calibration lists
calibrations_fileList.update(fnUpdateProductList(baseDataFolder_reduced, productList = calibrations_fileList))

# setup recipe "uves_cal_mflat"
clpRecipes[recipe] = fnSetRecipe(recipe,cfgFile)

# run recipe "uves_cal_predict"
clpResults[recipe] = clpRecipes[recipe](raw = {recipeKeywords[recipe]['TAG']:recipeFileLists[recipe] }, calib = calibrations_fileList, \
                                        output_dir = outputFolders[recipe], tmp_dir = tmpFolder)

clpResults[recipe] = None

print '{} complete.'.format(recipe)


In [ ]:
# 6) reduce spectra - uves_obs_scired

recipe = 'uves_obs_scired'

# get input file list
recipeFileLists[recipe] = fngetRecipeFileList(recipeName = recipe,\
                                              dataList = rawData_fileList,\
                                              keyword = recipeKeywords[recipe]['DPR_TYPE'])

# update calibration lists
calibrations_fileList.update(fnUpdateProductList(baseDataFolder_reduced, productList = calibrations_fileList))

# setup recipe "uves_cal_mflat"
clpRecipes[recipe] = fnSetRecipe(recipe,cfgFile)

# run recipe "uves_cal_predict"
for scienceFile in recipeFileLists[recipe]:
    clpResults[recipe] = clpRecipes[recipe](raw = {recipeKeywords[recipe]['TAG']:recipeFileLists[recipe] }, calib = calibrations_fileList, \
                                            output_dir = os.path.join(outputFolders[recipe], fits.getheader(scienceFile)['DATE-OBS']), tmp_dir = tmpFolder)